In [ ]:
import os
import time
import datetime
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, TensorDataset, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from utils_data import Vectorizer, HeadQA, HeadQA_IR, clean_words, parse_dataset, parse_ir_dataset, random_oversamplig, save_dataset_to_pickle, load_dataset_from_pickle
from training import train, validate, evaluate, train_ir, validate_ir, load_embeddings_from_file, make_embedding_matrix


import transformers
from transformers.optimization import AdamW
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer, BertModel, BertForMaskedLM

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from datasets import load_dataset

data_es = load_dataset('head_qa', 'es' )

In [ ]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [ ]:
#training_instances = parse_ir_dataset(training)
#validation_instances = parse_ir_dataset(validation)
#testing_instances = parse_ir_dataset(testing)

#oversampled_training = random_oversamplig(training_instances)

#save_dataset_to_pickle('../data/training_ir.pickle', training_instances)
#save_dataset_to_pickle('../data/validation_ir.pickle', validation_instances)
#save_dataset_to_pickle('../data/testing_ir.pickle', testing_instances)
#save_dataset_to_pickle('../data/oversampled_training_ir.pickle', oversampled_training)

In [ ]:
training_instances = load_dataset_from_pickle('../data/training_ir.pickle')
validation_instances = load_dataset_from_pickle('../data/validation_ir.pickle')
testing_instances = load_dataset_from_pickle('../data/testing_ir.pickle')
oversampled_training = load_dataset_from_pickle('../data/oversampled_training_ir.pickle')

In [ ]:
oversampled_training[0]

In [ ]:
vectorizer = Vectorizer.vectorize_ir_dataset(oversampled_training)

In [ ]:
vocab = vectorizer.sentence_vocab
label_vocab = vectorizer.label_vocab

In [ ]:
trainset = HeadQA_IR(instances=training_instances, vectorizer=vectorizer, right_padding=False, max_length=15)
validset = HeadQA_IR(instances=validation_instances, vectorizer=vectorizer, right_padding=False, max_length=15)
testset = HeadQA_IR(instances=testing_instances, vectorizer=vectorizer, right_padding=False, max_length=15)

In [ ]:
batch_size = 32
train_dt = DataLoader(trainset, batch_size=batch_size,drop_last=True)
valid_dt = DataLoader(validset, batch_size=batch_size,drop_last=True)
test_dt = DataLoader(testset, batch_size=batch_size,drop_last=True)

In [ ]:
class LSTM_QA(torch.nn.Module):
    def __init__(self, vocab_size, hidden_size, x_size, n_classes, embedding_size=300,
                 padding_idx=0, pretrained_embeddings=None): 
        super(LSTM_QA, self).__init__()
        self.embedding_size = embedding_size
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.n_classes = n_classes
        
        if pretrained_embeddings is None:
            self.emb = nn.Embedding(embedding_dim=self.embedding_size,num_embeddings=self.vocab_size,
                                    padding_idx=padding_idx)
        else:
            pretrained_embeddings = torch.from_numpy(pretrained_embeddings).float()
            self.emb = nn.Embedding(embedding_dim=self.embedding_size, num_embeddings=self.vocab_size,
                                    padding_idx=padding_idx, _weight=pretrained_embeddings)
            self.emb.weight.requires_grad = False
        
        self.dropout = nn.Dropout(0.5)
        self.lstm = nn.LSTM(embedding_size, hidden_size, batch_first=True, dropout=0.5,bidirectional=True)
        self.cosine = nn.CosineSimilarity(dim=1)
        self.linear = nn.Linear(self.hidden_size*2, self.n_classes)        
        
        
    def forward(self, x_0, x_1):
        x_0 = self.emb(x_0)
        x_1 = self.emb(x_1)
#         print(x_0.shape, x_1.shape)
        out_0, (ht_0, ct_0) = self.lstm(x_0)
        out_1, (ht_1, ct_1) = self.lstm(x_1)
#         print('out', out_0.shape, out_1.shape)
#         print('ht', ht_0.shape, ht_1.shape)
#         print('ct', ct_0.shape, ct_1.shape)
        x = self.cosine(out_0, out_1)
        x = self.linear(x)
        x = F.softmax(x, dim=0)
#         print(x.shape)
#         print(x)
        return x

In [ ]:
def get_optimizer(model, lr=0.01, wd=0.0):
    return torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

In [ ]:
# embedding_file = "trained_models/biomedical_embeddings/Scielo_wiki_FastText300.vec"
# word_to_idx, embeddings = load_embeddings_from_file(embedding_file)

# save_dataset_to_pickle('trained_models/biomedical_embeddings/word_to_index_ir.pickle', word_to_idx)
# save_dataset_to_pickle('trained_models/biomedical_embeddings/wordvectors_ir.pickle', embeddings)

In [ ]:
word_to_idx = load_dataset_from_pickle('trained_models/biomedical_embeddings/word_to_index_ir.pickle')
embeddings = load_dataset_from_pickle('trained_models/biomedical_embeddings/wordvectors_ir.pickle')

In [ ]:
embedding_file = "trained_models/biomedical_embeddings/Scielo_wiki_FastText300.vec"
words = vocab.vocab2index.keys()
embedding_matrix = make_embedding_matrix(embedding_file, list(words), word_to_idx, embeddings)

In [ ]:
model = LSTM_QA(vocab_size=len(vocab), hidden_size=64, x_size=trainset.max_length, n_classes=1, embedding_size=300,
               pretrained_embeddings=embedding_matrix)
optimizer = get_optimizer(model, lr = 0.001, wd = 1e-5)

In [ ]:
training_results = train_ir(model, optimizer, train_dt, valid_dt, validate_ir, epochs=50)